In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime
import os
import collections
import time

In [2]:
client = pymongo.MongoClient()
db = client['Fields']
path = r'C:\Users\ChastLai\Documents\Data\Ticks\\'
tick_list = os.listdir(path)
data = pd.DataFrame()
price = pd.DataFrame(db['收盤價'].find({'日期': {'$gte': datetime.datetime(2022, 9, 1), '$lte': datetime.datetime(2022, 10, 26)}}, {'_id': 0})).set_index('日期')
priceo = pd.DataFrame(db['開盤價'].find({'日期': {'$gte': datetime.datetime(2022, 9, 1), '$lte': datetime.datetime(2022, 10, 26)}}, {'_id': 0})).set_index('日期')
price = (price / priceo - 1).shift(-1)
price.columns = price.columns.astype(int)

In [52]:
total_, iso_, nis_, nis_s_, nis_c_, nis_b_ = [], [], [], [], [], []
df_ = pd.DataFrame()
def calculator(df_, tick):
    ctime = time.time()
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    pf = price.loc[stk_test.index[0].strftime('%Y-%m-%d')]

    ms = 0.1
    total = {x: [0, 0] for x in stk_test.code.unique()}
    iso = {x: [0, 0] for x in stk_test.code.unique()}
    nis = {x: [0, 0] for x in stk_test.code.unique()}
    nis_s = {x: [0, 0] for x in stk_test.code.unique()}
    nis_c = {x: [0, 0] for x in stk_test.code.unique()}
    nis_b = {x: [0, 0] for x in stk_test.code.unique()}

    
    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    for _, v in stk_test[~(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        if v['tick_type'] == 1:
            iso[v.code][0] += v['volume']
            total[v.code][0] += v['volume']
        elif v['tick_type'] == 2:
            iso[v.code][1] += v['volume']
            total[v.code][1] += v['volume']
    for i, v in stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))].iterrows():
        time_package = [i - datetime.timedelta(milliseconds=ms), i + datetime.timedelta(milliseconds=ms)]
        nis_package = stk_test[(stk_test.index > time_package[0]) & (stk_test.index <= time_package[1])].groupby(['ts', 'code']).sum()
        if len(nis_package) == 1:
            if v['tick_type'] == 1: # 外盤成交
                iso[v.code][0] += v['volume']
                total[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                iso[v.code][1] += v['volume']
                total[v.code][1] += v['volume']
        elif len(nis_package.index.get_level_values(1).unique()) == 1:
            if v['tick_type'] == 1: # 外盤成交
                nis_s[v.code][0] += v['volume']
                total[v.code][0] += v['volume']
                nis[v.code][0] += v['volume']
            elif v['tick_type'] == 2: # 內盤成交
                nis_s[v.code][1] += v['volume']
                total[v.code][1] += v['volume']
                nis[v.code][1] += v['volume']
        else:
            if len(nis_package[nis_package.index.get_level_values(1) == v['code']]) == 1:
                if v['tick_type'] == 1: # 外盤成交
                    nis_c[v.code][0] += v['volume']
                    total[v.code][0] += v['volume']
                    nis[v.code][0] += v['volume']
                elif v['tick_type'] == 2: # 內盤成交
                    nis_c[v.code][1] += v['volume']
                    total[v.code][1] += v['volume']
                    nis[v.code][1] += v['volume']
            else:
                if v['tick_type'] == 1: # 外盤成交
                    nis_b[v.code][0] += v['volume']
                    total[v.code][0] += v['volume']
                    nis[v.code][0] += v['volume']
                elif v['tick_type'] == 2: # 內盤成交
                    nis_b[v.code][1] += v['volume']
                    total[v.code][1] += v['volume']
                    nis[v.code][1] += v['volume']
    print(tick.split('.')[0])
    for k, l in {'total': total, 'iso': iso, 'nis': nis, 'nis_s': nis_s, 'nis_c': nis_c, 'nis_b': nis_b}.items():
        pf1 = pf.loc[l.keys()]
        pf1.name = 'returns'
        se = pd.Series({key: (v[0]-v[1]) / (v[0]+v[1]) if (v[0] + v[1]) != 0 else 0 for key, v in l.items()})
        se.name = 'coi'
        d = pd.concat([se, pf1], axis=1)#.corr().iloc[0, 1]
        d.reset_index(inplace=True)
        d = pd.concat([d, pd.Series([k]*len(d), name='Type')], axis=1)
        df_ = pd.concat([df_, d], axis=0)
        globals()[k + '_'].append(d.corr().iloc[1, 2])
        print(k, '{:.2f}'.format(d.corr().iloc[1, 2]))
    print('耗時: ', '{:.2f}'.format(time.time() - ctime))
    return df_
for tick in tick_list:
    df_ = calculator(df_, tick)


2022-09-01
total 0.25
iso 0.18
nis 0.31
nis_s 0.14
nis_c 0.27
nis_b 0.17
耗時:  76.91
2022-09-02
total -0.37
iso -0.40
nis -0.23
nis_s -0.22
nis_c 0.20
nis_b -0.10
耗時:  103.23
2022-09-05
total 0.30
iso 0.23
nis 0.27
nis_s 0.02
nis_c 0.10
nis_b 0.06
耗時:  76.74
2022-09-06
total -0.01
iso 0.02
nis -0.19
nis_s -0.21
nis_c -0.09
nis_b 0.12
耗時:  67.02
2022-09-07
total -0.24
iso -0.30
nis 0.17
nis_s 0.08
nis_c 0.06
nis_b 0.03
耗時:  71.22
2022-09-08
total 0.00
iso 0.06
nis -0.21
nis_s -0.16
nis_c -0.24
nis_b -0.30
耗時:  53.09
2022-09-12
total 0.14
iso 0.03
nis 0.30
nis_s 0.22
nis_c 0.32
nis_b 0.15
耗時:  31.26
2022-09-13
total -0.36
iso -0.32
nis -0.38
nis_s -0.32
nis_c -0.16
nis_b 0.12
耗時:  31.55
2022-09-14
total -0.35
iso -0.28
nis -0.18
nis_s -0.15
nis_c -0.12
nis_b 0.32
耗時:  47.33
2022-09-15
total 0.28
iso 0.27
nis -0.00
nis_s 0.03
nis_c 0.11
nis_b -0.12
耗時:  32.44
2022-09-16
total -0.16
iso -0.19
nis 0.04
nis_s 0.02
nis_c -0.03
nis_b -0.17
耗時:  39.73
2022-09-19
total -0.07
iso -0.10
nis -0.06
n

In [51]:
df_

,index,coi,returns
0,2498,-0.161870,0.056198
1,3443,-0.279641,0.030418
2,2888,-0.223402,-0.015046
3,2027,-0.443005,-0.015524
4,3406,-0.100957,0.008949
...,...,...,...
10747,2603,0.355014,0.007246
10748,3443,0.124357,-0.013614
10749,2474,0.113490,0.018576
10750,2388,-0.181385,-0.035541


In [31]:
ms = 100
for tick in tick_list:
    stk_test = pd.read_json(path + tick)
    stk_test['ts'] = pd.to_datetime(stk_test['ts'])
    stk_test.set_index('ts', inplace=True)
    stk_test['td'] = stk_test.index
    stk_test.td = stk_test.td.diff()
    print(tick)
    print('iso_len: ', len(stk_test[~(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))
    print('nis_len: ', len(stk_test[(stk_test['td'] < datetime.timedelta(milliseconds=ms))]))

2022-09-01.json
iso_len:  192338
nis_len:  47572
2022-09-02.json
iso_len:  203630
nis_len:  53000
2022-09-05.json
iso_len:  159038
nis_len:  37177
2022-09-06.json
iso_len:  162272
nis_len:  33332
2022-09-07.json
iso_len:  153871
nis_len:  37283
2022-09-08.json
iso_len:  137825
nis_len:  32462
2022-09-12.json
iso_len:  147739
nis_len:  34889
2022-09-13.json
iso_len:  149964
nis_len:  36793
2022-09-14.json
iso_len:  172461
nis_len:  31745
2022-09-15.json
iso_len:  157423
nis_len:  30601
2022-09-16.json
iso_len:  150545
nis_len:  27736
2022-09-19.json
iso_len:  149136
nis_len:  29460
2022-09-20.json
iso_len:  164748
nis_len:  32943
2022-09-21.json
iso_len:  202163
nis_len:  79345
2022-09-22.json
iso_len:  197715
nis_len:  45887
2022-09-23.json
iso_len:  172529
nis_len:  37619
2022-09-26.json
iso_len:  194841
nis_len:  38171
2022-09-27.json
iso_len:  167639
nis_len:  34763
2022-09-28.json
iso_len:  206242
nis_len:  45836
2022-09-29.json
iso_len:  181904
nis_len:  43723
2022-09-30.json
iso_